In [34]:
belasting_percentage_laag = 0.371
belasting_percentage_hoog = 0.495
belasting_top_bedrag = 68508

jaar = 12
vakantiegeld_13e_maand = 13.96
looptijd_hypotheek = 360 # in maanden
hypotheek_bedrag = 145000
rente_percentage = 0.015

gemiddelde_loonstijging_percentage = 1.03

algemene_heffingskorting_ondergrens = 21043
algemene_heffingskorting_middengrens = 68507
algemene_heffingskorting_middengrens_percentage = 0.05977
algemene_heffingskorting_ondergrens_bedrag = 2837

woz_waarde = 17400
woz_waardes = [127000, 132000, 133000, 132000,135000, 153000, 174000]
woz_groei = 7.8
taxatie_waarde = 202000


maandsalaris_bruto = 3200
totaal_jaarsalaris = maandsalaris_bruto * vakantiegeld_13e_maand

pensioen_heffing = 138
pensioen_heffing_jaar = pensioen_heffing * jaar
totaal_jaarsalaris_exclusief = (maandsalaris_bruto - pensioen_heffing) * jaar
import pandas as pd


In [6]:
def bereken_heffingskorting(totaal_inkomen):
    if totaal_inkomen <= algemene_heffingskorting_ondergrens:
        return algemene_heffingskorting_ondergrens_bedrag
    elif totaal_inkomen <= algemene_heffingskorting_middengrens:
        return algemene_heffingskorting_ondergrens_bedrag - (algemene_heffingskorting_middengrens_percentage * (totaal_inkomen - algemene_heffingskorting_ondergrens))
    else:
        return 0

In [7]:
def bereken_arbeidskorting(arbeids_inkomen):
    arbeidskorting_beginbedrag = 4205
    arbeidskorting_percentage = 0.06
    arbeidskorting_grens = 35652
    return arbeidskorting_beginbedrag - arbeidskorting_percentage * (arbeids_inkomen - arbeidskorting_grens)


In [214]:
def bereken_belasting(totaal_inkomen):
    belasting_bedrag = 0
    schijf_1_belasting = 0
    schijf_2_belasting = 0
    if totaal_inkomen >= belasting_top_bedrag:
        schijf_1_belasting = belasting_top_bedrag * belasting_percentage_laag
        schijf_2_belasting = (totaal_inkomen - belasting_top_bedrag) * belasting_percentage_hoog
        belasting_bedrag = schijf_1_belasting + schijf_2_belasting
    else:
        belasting_bedrag = totaal_inkomen * belasting_percentage_laag
        schijf_1_belasting = belasting_bedrag
    return [belasting_bedrag, schijf_1_belasting, schijf_2_belasting]

In [217]:
def bereken_netto_salaris(salaris, aftrekposten, jaar, woning=True):
    belastbaar_inkomen = salaris - aftrekposten
    arbeidskorting_bedrag = bereken_arbeidskorting(salaris)
    heffingskorting_bedrag = bereken_heffingskorting(salaris)
    if woning:
        forfait = bereken_woz_waarde(woz_waarde, jaar) * 0.05
        belastbaar_inkomen = belastbaar_inkomen + forfait
    belasting_bedrag_totaal = bereken_belasting(belastbaar_inkomen)
    belasting_bedrag_met_kortingen = belasting_bedrag_totaal[0] - arbeidskorting_bedrag - heffingskorting_bedrag
    return [salaris - belasting_bedrag_met_kortingen, belasting_bedrag_totaal[0], aftrekposten, belasting_bedrag_totaal[1], belasting_bedrag_totaal[2]]

In [218]:
bereken_netto_salaris(42684, 2000,  1, False)

[32916.83343, 15093.764, 2000, 15093.764, 0]

In [219]:
def bereken_woz_waarde(woz_waarde, jaar):
    return woz_waarde + (jaar * woz_groei)

In [305]:
def bereken_annuiteit(totaal_bedrag, rente_percentage, resterende_tijd = 360):
    maandrente = rente_percentage / 12
    return (maandrente / (1 - ((1 + maandrente) ** -resterende_tijd))) * totaal_bedrag

In [306]:
def bereken_aflosvrij_hypotheek(totaal_bedrag, rente_percentage):
    return totaal_bedrag * rente_percentage
        

In [307]:
def bereken_woonlastenwijzer(restsom, rente_vast, rente):
    taxatie_waarde = 202000
    restschuld_percentage = (restsom / taxatie_waarde) * 100
    if rente_vast == 10:
        if restschuld_percentage <= 55:
            return 0.0106 
        elif restschuld_percentage <= 60:
            return 0.0109 
        elif restschuld_percentage <= 65:
            return 0.0111 
        elif restschuld_percentage <= 70:
            return 0.0113
        elif restschuld_percentage <= 75:
            return 0.0117
    elif rente_vast == 20:
        if restschuld_percentage <= 55:
            return 0.0143 
        elif restschuld_percentage <= 60:
            return 0.0146 
        elif restschuld_percentage <= 65:
            return 0.0148 
        elif restschuld_percentage <= 70:
            return 0.0150
        elif restschuld_percentage <= 75:
            return 0.0154
    else:
        return rente
    

In [308]:
# annuit hypotheek

def bereken_annuit_hypotheek(totaal_bedrag, rente_percentage, rentevaste_periode=30, rente_percentage_variabel = 0, verstrekker='overig'):
    rentevaste_maanden = rentevaste_periode * 12
    annuiteit = bereken_annuiteit(totaal_bedrag, rente_percentage)
    rente_jaren_dict = {}
    jaren = 0
    rentebedrag_jaar = 0
    actuele_rente = 0
    for i in range(360):
        x = i + 1
        if i < rentevaste_maanden:
            if verstrekker == 'ING':
                actuele_rente = (bereken_woonlastenwijzer(totaal_bedrag, rentevaste_periode, rente_percentage))
                rente = (totaal_bedrag * actuele_rente) / 12
                annuiteit = bereken_annuiteit(totaal_bedrag, actuele_rente, 360-x)
            else:
                rente = (totaal_bedrag * rente_percentage) / 12
        else:
            rente = (totaal_bedrag * rente_percentage_variabel) / 12
            actuele_rente = rente_percentage_variabel
            annuiteit = bereken_annuiteit(totaal_bedrag, actuele_rente, 360-x)
        rentebedrag_jaar += rente
        aflossing = annuiteit - rente
        totaal_bedrag = totaal_bedrag - aflossing
        if x % 12 == 0:
            jaren += 1
            rente_jaren_dict[jaren] = [rentebedrag_jaar, totaal_bedrag, actuele_rente, aflossing]
            rentebedrag_jaar = 0

        # print('maand', x, 'totaalbedrag', "{:.0f}".format(totaal_bedrag), 'aflossing', "{:.0f}".format(aflossing), 'rente', "{:.0f}".format(rente), 'maandbedrag', "{:.0f}".format(aflossing+rente))
    return rente_jaren_dict





In [309]:
rente_bedragen_annuiteit_20_vast_ing = bereken_annuit_hypotheek(hypotheek_bedrag, 0.0154, 20, 0.05, 'ING')
rente_bedragen_annuiteit_20_vast_ing_values = [el[0] for el in rente_bedragen_annuiteit_20_vast_ing.values()]
totaal_bedrag_aan_rente_20_vast_ing = sum(rente_bedragen_annuiteit_20_vast_ing_values)
totaal_bedrag_aan_rente_20_vast_ing


ZeroDivisionError: float division by zero

In [310]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

def bereken_netto_salaris_en_belasting(hypotheek_bedrag, rente_percentage, hoeveel_jaar, type_hypotheek, rentevaste_periode = 30, rente_na_vaste = 0, verstrekker = 'overig'):
    bruto_salaris = totaal_jaarsalaris
    kolommen = ['Label', 'Hypotheektype', 'Hypotheekbedrag', 'Rentepercentage begin', 'Rentepercentage actueel', 'Restschuld', 'Aflossing','Rentevaste periode', 'Verstrekker', 'Jaar', 'Datum', 'Bruto', 'Netto', 'Bedrag kwijt', 'Percentage kwijt', 'Jaarlasten hypotheek', 'Netto salaris na hypotheeklasten', 'Aftrekposten', 'Belasting totaal', 'Schijf 1 belasting', 'Schijf 2 belasting']
    dataframe = pd.DataFrame(columns=kolommen)
    for i in range(hoeveel_jaar):
        jaar = i + 1
        jaar_lasten = 0
        nu = datetime.now()
        nieuwe_datum = (nu + relativedelta(years=jaar)).strftime("%Y-%m-%d")
        if type_hypotheek == 'aflosvrij':
            netto_salaris = bereken_netto_salaris(bruto_salaris, 0, jaar)
            jaar_lasten = bereken_aflosvrij_hypotheek(hypotheek_bedrag, rente_percentage)
        elif type_hypotheek == 'annuit':
            rente_bedragen_annuiteit = bereken_annuit_hypotheek(hypotheek_bedrag, rente_percentage, rentevaste_periode, rente_na_vaste, verstrekker)
            rest_schuld = rente_bedragen_annuiteit[jaar][1]
            actuele_rente = rente_bedragen_annuiteit[jaar][2]
            netto_salaris = bereken_netto_salaris(bruto_salaris, rente_bedragen_annuiteit[jaar][0], jaar)
            jaar_lasten = bereken_annuiteit(hypotheek_bedrag, rente_percentage) * 12
        
        belasting_bedrag = netto_salaris[0] - bruto_salaris
        belasting_percentage = 100 - ((netto_salaris[0] / bruto_salaris) * 100)
        rentepercentage_tekst = "{:.2%}".format(rente_percentage)
        rij = {'Hypotheektype': type_hypotheek,
        'Hypotheekbedrag' : hypotheek_bedrag,
         'Jaar': jaar, 
         'Datum': nieuwe_datum,
         'Rentepercentage begin' : rentepercentage_tekst,
         'Rentepercentage actueel' : "{:.2%}".format(actuele_rente),
         'Restschuld' : "{:.2f}".format(rest_schuld),
         'Rentevaste periode' : rentevaste_periode,
         'Verstrekker' : verstrekker,
         'Label' :  f'{verstrekker} {rentevaste_periode} {rentepercentage_tekst}',
         'Bruto': float("{:.2f}".format(bruto_salaris)), 
         'Netto': float("{:.2f}".format(netto_salaris[0])),
         'Bedrag kwijt' : float("{:.2f}".format(belasting_bedrag)),
         'Percentage kwijt' : float("{:.2f}".format(belasting_percentage)),
         'Jaarlasten hypotheek' : float("{:.2f}".format(jaar_lasten)),
         'Netto salaris na hypotheeklasten' : float("{:.2f}".format(netto_salaris[0] - jaar_lasten)),
         'Belasting totaal': "{:.2f}".format(netto_salaris[1]),
         'Aftrekposten' : "{:.2f}".format(netto_salaris[2]),
         'Schijf 1 belasting': "{:.2f}".format(netto_salaris[3]),
          'Schijf 2 belasting': "{:.2f}".format(netto_salaris[4]),
          'Aflossing' : "{:.2f}".format(rente_bedragen_annuiteit[jaar][0])
         }
        dataframe = dataframe.append(rij, ignore_index = True)
        bruto_salaris = bruto_salaris * gemiddelde_loonstijging_percentage
    return dataframe
        
        

# print('Jaar', jaar, 'Bruto',"{:.0f}".format(bruto_salaris), 'Netto', "{:.0f}".format(netto_salaris), 'Bedrag kwijt',"{:.0f}".format(belasting_bedrag), 'Percentage kwijt', "{:.2f}".format(belasting_percentage), 'Jaarlasten hypotheek', "{:.0f}".format(jaar_lasten), "Netto salaris na hypotheeklasten", "{:.0f}".format(netto_salaris - jaar_lasten))
#nog berekenen met de maandlasten wat ik dan precies kwijt ben


In [311]:
df_10_annuit_ing =  bereken_netto_salaris_en_belasting(145000, 0.0117, 30, 'annuit', 10, 0.05, verstrekker ='ING')

ZeroDivisionError: float division by zero

In [300]:
df_20_annuit_ing = bereken_netto_salaris_en_belasting(145000, 0.0154, 30, 'annuit', 20, 0.05, verstrekker= 'ING')

NameError: name 'oorspronk_bedrag' is not defined

In [290]:
df_30_annuit_venn = bereken_netto_salaris_en_belasting(145000, 0.0184, 30, 'annuit', 30, 0, verstrekker= 'Venn')

In [291]:
df_30_annuit_munt = bereken_netto_salaris_en_belasting(145000, 0.0175, 30, 'annuit', 30, 0, verstrekker= 'Munt')

In [292]:
df_30_annuit_lloyds = bereken_netto_salaris_en_belasting(145000, 0.0173, 30, 'annuit', 30, 0, verstrekker= 'Lloyds')

In [293]:
hypotheken = [df_10_annuit_ing, df_20_annuit_ing, df_30_annuit_venn, df_30_annuit_munt, df_30_annuit_lloyds]

In [294]:
alle_dfs2 = pd.concat(hypotheken)

In [295]:
alle_dfs2.to_csv('hypotheek.csv', sep=';')

In [ ]:
[sum(x["Netto salaris na hypotheeklasten"]) for x in ]

In [195]:
#verschil over 20 jaar
a = df_10_annuit_ing[df_10_annuit_ing["Jaar"] <= 10]['Netto salaris na hypotheeklasten']
b = df_20_annuit_ing[df_20_annuit_ing["Jaar"] <= 10]['Netto salaris na hypotheeklasten']

sum([a_i - b_i for a_i, b_i in zip(a, b)])

1288.890000000003

In [196]:
a = df_20_annuit_ing[df_20_annuit_ing["Jaar"] <= 30]['Netto salaris na hypotheeklasten']
b = df_30_annuit_venn[df_30_annuit_venn["Jaar"] <= 30]['Netto salaris na hypotheeklasten']
sum([a_i - b_i for a_i, b_i in zip(a, b)])

4319.690000000028

In [630]:
alle_dfs = pd.DataFrame()
rente_percentages = [x * 0.00075 for x in range(18, 30)]
for x in rente_percentages:
    alle_dfs =alle_dfs.append(bereken_netto_salaris_en_belasting(145000, x, 30, 'annuit', 20, 0.05))

rente_percentages_10 = [y * 0.00075 for y in range(13, 20)]
for y in rente_percentages_10:
    alle_dfs = alle_dfs.append(bereken_netto_salaris_en_belasting(145000, y, 30, 'annuit', 10, 0.05))

In [631]:
alle_dfs.to_csv('hypotheek2.csv', sep=';')

In [633]:
# rente_percentages = [x * 0.00075 for x in range(13, 30)]
# for x in rente_percentages:
#     rente_bedragen_annuiteit_30_vast = bereken_annuit_hypotheek(hypotheek_bedrag, x)
#     totaal_bedrag_aan_rente_30_vast = sum(rente_bedragen_annuiteit_30_vast.values())
#     print('Rentepercentage', "{:.2f}".format(x * 100), "Totaal rente", "{:.0f}".format(totaal_bedrag_aan_rente_30_vast))

Rentepercentage 0.97 Totaal rente 22297
Rentepercentage 1.05 Totaal rente 24097
Rentepercentage 1.12 Totaal rente 25910
Rentepercentage 1.20 Totaal rente 27734
Rentepercentage 1.28 Totaal rente 29571
Rentepercentage 1.35 Totaal rente 31420
Rentepercentage 1.43 Totaal rente 33280
Rentepercentage 1.50 Totaal rente 35153
Rentepercentage 1.57 Totaal rente 37037
Rentepercentage 1.65 Totaal rente 38934
Rentepercentage 1.73 Totaal rente 40842
Rentepercentage 1.80 Totaal rente 42763
Rentepercentage 1.88 Totaal rente 44695
Rentepercentage 1.95 Totaal rente 46639
Rentepercentage 2.02 Totaal rente 48595
Rentepercentage 2.10 Totaal rente 50562
Rentepercentage 2.18 Totaal rente 52541
